In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install Packages

In [ ]:
!pip install --quiet mwparserfromhell
!pip install --quiet wikipedia
!pip install --quiet datasets==3.6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.6 MB/s eta 0:00:00


# Get Dataset

In [ ]:
# https://huggingface.co/datasets/sabilmakbar/indo_wiki

from datasets import load_dataset

dataset = load_dataset(
    "sabilmakbar/indo_wiki",
    "indowiki_all",
    trust_remote_code=True
)

README.md: 0.00B [00:00, ?B/s]

indo_wiki.py: 0.00B [00:00, ?B/s]

wiki_ace_20230901_raw_dataset.csv:   0%|          | 0.00/4.87M [00:00<?, ?B/s]

wiki_ban_20230901_raw_dataset.csv:   0%|          | 0.00/17.6M [00:00<?, ?B/s]

wiki_bjn_20230901_raw_dataset.csv:   0%|          | 0.00/6.67M [00:00<?, ?B/s]

...

Generating min split:   0%|          | 0/227024 [00:00<?, ? examples/s]

Generating ms split:   0%|          | 0/367463 [00:00<?, ? examples/s]

Generating nia split:   0%|          | 0/1651 [00:00<?, ? examples/s]

Generating su split:   0%|          | 0/61557 [00:00<?, ? examples/s]

Generating tet split:   0%|          | 0/1465 [00:00<?, ? examples/s]

In [ ]:
# only get target datasets

from datasets import DatasetDict

langs_to_keep = ["id", "jv", "su", "ban", "ace", "bug"] # need eng, bbc, mad
dataset = DatasetDict({lang: dataset[lang] for lang in langs_to_keep})

## Scrape Additional Dataset

In [ ]:
from datasets import Dataset

def scrape(lang_id=None, date_ver=None, dset_name="wikipedia"):
  '''
  Script on Generating Wikipedia Data that are dumped into https://dumps.wikimedia.org/
  More info can be read on https://huggingface.co/datasets/wikipedia
  -------------------
  Check here to see available indexed data: https://dumps.wikimedia.org/backup-index.html
  Also check here to see language meta from its code: https://meta.wikimedia.org/wiki/List_of_Wikipedias
  '''

  print("Loading the dataset from Wikipedia...")
  if lang_id and date_ver:
    df = load_dataset(dset_name, language=lang_id, date=date_ver, split="train", trust_remote_code=True).to_pandas()
  else:
    df = load_dataset(dset_name, "20220301.en", split="train", streaming=True, trust_remote_code=True)
    df = pd.DataFrame([x for _, x in zip(range(4000), df)])

  print("Loading done!")
  print(f"Data collected: {df.shape[0]}")

  dataset = Dataset.from_pandas(df).remove_columns("id")

  return dataset

In [ ]:
# Scrape Batak Toba (bbc)
dataset_bbc = scrape(lang_id="bbc", date_ver="20250801")
dataset["bbc"] = dataset_bbc

Loading the dataset from Wikipedia...
Loading done!
Data collected: 1159


In [ ]:
# Scrape Madurese (mad)
dataset_mad = scrape(lang_id="mad", date_ver="20250801")
dataset["mad"] = dataset_mad

Loading the dataset from Wikipedia...
Loading done!
Data collected: 2206


In [ ]:
# Scrape English (eng)
dataset_eng = scrape()
dataset["eng"] = dataset_eng

Loading the dataset from Wikipedia...
Loading done!
Data collected: 4000


In [ ]:
print(dataset)

DatasetDict({
    id: Dataset({
        features: ['url', 'title', 'text'],
        num_rows: 657990
    })
    jv: Dataset({
        features: ['url', 'title', 'text'],
        num_rows: 73150
    })
    su: Dataset({
        features: ['url', 'title', 'text'],
        num_rows: 61557
    })
    ban: Dataset({
        features: ['url', 'title', 'text'],
        num_rows: 20243
    })
    ace: Dataset({
        features: ['url', 'title', 'text'],
        num_rows: 12932
    })
    bug: Dataset({
        features: ['url', 'title', 'text'],
        num_rows: 15877
    })
    bbc: Dataset({
        features: ['url', 'title', 'text'],
        num_rows: 1159
    })
    mad: Dataset({
        features: ['url', 'title', 'text'],
        num_rows: 2206
    })
    eng: Dataset({
        features: ['url', 'title', 'text'],
        num_rows: 4000
    })
})


## Get Summary from Titles

In [ ]:
import os
os.makedirs("data", exist_ok=True)

In [ ]:
language_codes = {
    "id": "ind",
    "jv": "jav",
    "su": "sun",
    "ban": "ban",
    "bbc": "bbc",
    "mad": "mad",
    "ace": "ace",
    "bug": "bug",
}

In [ ]:
# import time
# import random
# import wikipedia
# import pandas as pd
# from datasets import load_dataset
# import os
# from concurrent.futures import ThreadPoolExecutor, as_completed
# from wikipedia.exceptions import DisambiguationError, PageError

# language_codes = {
#     "id": "ind",
#     "jv": "jav",
#     "su": "sun",
#     "ban": "ban",
#     "bbc": "bbc",
#     "mad": "mad",
#     "ace": "ace",
#     "bug": "bug",
# }

# # # Check if article was created by human (not a bot)
# # def is_human_created(title, lang):
# #     url = f"https://{lang}.wikipedia.org/w/api.php"
# #     params = {
# #         "action": "query",
# #         "prop": "revisions",
# #         "rvlimit": 1,
# #         "rvdir": "newer",
# #         "titles": title,
# #         "format": "json"
# #     }
# #     try:
# #         response = requests.get(url, params=params, timeout=10)
# #         data = response.json()
# #         page = next(iter(data["query"]["pages"].values()))
# #         user = page["revisions"][0]["user"]
# #         return not user.lower().endswith("bot")
# #     except:
# #         return False

# # Function to fetch summary
# def fetch_summary(title, lang, label):
#     wikipedia.set_lang(lang)
#     try:
#         if lang == "bbc":
#             # Fetch 6 sentences
#             summary = wikipedia.summary(title, sentences=6)
#             # Split into sentences
#             sentences = summary.split(". ")
#             # Ensure we have at least 6 sentences
#             sentences = [s.strip() for s in sentences if s.strip()]
#             part1 = ". ".join(sentences[:2])
#             part2 = ". ".join(sentences[2:4])
#             part3 = ". ".join(sentences[4:6])
#             results = []
#             if part1:
#                 results.append({"text": part1, "label": label})
#             if part2:
#                 results.append({"text": part2, "label": label})
#             if part3:
#                 results.append({"text": part3, "label": label})
#             return results
#         else:
#             summary = wikipedia.summary(title, sentences=3)
#             return [{"text": summary, "label": label}]
#     except (DisambiguationError, PageError):
#         return []
#     except Exception:
#         return []

# # Loop through each language
# for lang, label in language_codes.items():
#     print(f"\nProcessing language: {lang} → {label}")

#     # Get and shuffle titles
#     sample_size = min(3500, len(dataset[lang]))
#     data = dataset[lang].shuffle(seed=42).select(range(sample_size))
#     titles = data["title"]

#     results = []
#     with ThreadPoolExecutor(max_workers=10) as executor:
#         futures = {executor.submit(fetch_summary, title, lang, label): title for title in titles}

#         for future in as_completed(futures):
#             result = future.result()
#             if result:
#                 results.extend(result)

#             if len(results) >= 3500:
#                 break

#     print(f"Collected {len(results)} summaries for {label}")

#     # Save to CSV
#     df = pd.DataFrame(results)
#     output_path = f"data/{label}.csv"
#     df.to_csv(output_path, index=False)
#     print(f"Saved to {output_path}")



Processing language: bbc → bbc
Collected 1988 summaries for bbc
Saved to data/bbc.csv


# Data Cleaning

In [ ]:
import pandas as pd
from glob import glob

# Path to your CSV folder
csv_folder = "/content/drive/MyDrive/04_language detection/indo_datasets"

# Find all CSV files
csv_files = glob(os.path.join(csv_folder, "*.csv"))

dfs = [pd.read_csv(f) for f in csv_files]
df_all = pd.concat(dfs, ignore_index=True)
df_all["label"].value_counts()

,count
label,
bug,3489
ban,3488
ace,3487
sun,3481
jav,3458
ind,3451
mad,2202
bbc,1988


## Parsing

In [ ]:
!pip install --quiet regex

In [ ]:
import re
import regex

# # Validate each row has exactly 3 sentences
# def validate_three_sentences(text: str) -> bool:
#     """
#     Returns True if the text has exactly 3 sentences, else False.
#     A sentence ends with '.', '!', or '?'.
#     """
#     sentences = re.split(r'[.!?]+', text.strip())
#     sentences = [s for s in sentences if s.strip()]  # remove empties
#     return len(sentences) == 3


def remove_newlines(text: str) -> str:
    if not isinstance(text, str):
        return ""
    return text.replace("\n", "")

def remove_parentheses(text: str) -> str:
    """
    Removes any content inside () including the parentheses.
    """
    return re.sub(r'\([^)]*\)', '', text)

def remove_equals(text: str) -> str:
    return text.replace("==", "")

def remove_doubledashes(text: str) -> str:
    return text.replace("--", "")

def remove_links(text: str) -> str:
    if not isinstance(text, str):
        return ""
    # matches http:// or https:// followed by non-space characters
    return re.sub(r"http\S+|www\S+", "", text)

def keep_latin_only(text: str) -> str:
    """
    Keeps only Latin script characters (with diacritics),
    spaces, and common punctuation. Removes all non-Latin scripts.
    """
    # \p{Latin} matches all Latin characters including diacritics
    return regex.sub(r"[^\p{Latin}\s.,;:!?'\-–—()/]", "", text)

## remove sentences that are too similar


In [ ]:
df_parsed = df_all[["label"]].copy()

df_parsed["text"] = (
    df_all["text"]
    .apply(lambda x: remove_newlines(x))
    .apply(lambda x: remove_parentheses(x))
    .apply(lambda x: remove_equals(x))
    .apply(lambda x: remove_doubledashes(x))
    .apply(lambda x: remove_links(x))
    .apply(lambda x: keep_latin_only(x))
)

df_parsed

,label,text
0,bug,Grandchamp iyanaritu séuwa komun ri déparetema...
1,bug,Baratier iyanaritu séuwa komun déparetema Haut...
2,bug,Prémontré iyanaritu séuwa komun déparetema Ais...
3,bug,Montrevault iyanaritu séuwa komun ri déparetem...
4,bug,Saint-Loup-en-Comminges iyanaritu séuwa komun ...
...,...,...
25039,ind,Exhyalanthrax uroganus adalah spesies lalat ya...
25040,ind,"Victor Prouvé, adalah seorang pelukis, pematu..."
25041,ind,Discovery of Love adalah serial televisi Kore...
25042,ind,The Darjeeling Limited adalah film drama komed...


## Deduplication

In [ ]:
df_parsed = df_parsed.drop_duplicates(subset=["text"])
df_parsed.describe()

,label,text
count,24285,24285
unique,8,24285
top,bug,Cibal Barat adalah sebuah Kecamatan di Kabupat...
freq,3483,1


## Remove NaN

In [ ]:
nan_count = df_parsed["text"].isna().sum()
print("Number of NaN rows in text column:", nan_count)

nan_rows = df_parsed[df_parsed["text"].isna()]
print(nan_rows)

df_parsed = df_parsed.dropna()

Number of NaN rows in text column: 0
Empty DataFrame
Columns: [label, text]
Index: []


## Export

In [ ]:
df_parsed["label"].value_counts()

,count
label,
bug,3483
ace,3476
ind,3412
ban,3404
jav,3363
sun,3055
mad,2189
bbc,1903


In [ ]:
df_parsed.to_csv("/content/data/all.csv", index=False)
df_parsed

,label,text
0,bug,Grandchamp iyanaritu séuwa komun ri déparetema...
1,bug,Baratier iyanaritu séuwa komun déparetema Haut...
2,bug,Prémontré iyanaritu séuwa komun déparetema Ais...
3,bug,Montrevault iyanaritu séuwa komun ri déparetem...
4,bug,Saint-Loup-en-Comminges iyanaritu séuwa komun ...
...,...,...
25039,ind,Exhyalanthrax uroganus adalah spesies lalat ya...
25040,ind,"Victor Prouvé, adalah seorang pelukis, pematu..."
25041,ind,Discovery of Love adalah serial televisi Kore...
25042,ind,The Darjeeling Limited adalah film drama komed...
